# 🎵 Konser Salonu C80 Akustik Parametresi Tahmini

## TÜBİTAK 1001 Projesi - Final Model

**Amaç:** 63 ölçülmüş koltuktan 120 bilinmeyen koltuk için C80 tahmini

**Model:** Bidirectional RNN + Ensemble Deep Learning

---

In [ ]:
# =============================================================================
# HÜCRE 1: KÜTÜPHANELER VE AYARLAR
# =============================================================================

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist
import time
import warnings
warnings.filterwarnings('ignore')

# Görselleştirme ayarları
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
sns.set_style('whitegrid')

# GPU kontrolü
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("=" * 70)
print("🎵 KONSER SALONU C80 TAHMİN MODELİ - FINAL")
print("=" * 70)
print(f"   🖥️ Cihaz: {device.upper()}")
if device == 'cuda':
    print(f"   🎮 GPU: {torch.cuda.get_device_name(0)}")
print(f"   📦 PyTorch: {torch.__version__}")

In [ ]:
# =============================================================================
# HÜCRE 2: VERİ YÜKLEME VE ÖN İŞLEME
# =============================================================================

print("\n" + "=" * 70)
print("📊 VERİ YÜKLEME")
print("=" * 70)

# Veri yükle
df = pd.read_excel("salon.xlsx")
df = df.rename(columns={"Unnamed: 0": "seat_id"})

# Frekans bantları
FREQS = [125, 250, 500, 1000, 2000, 4000]
FREQ_LABELS = ['125 Hz', '250 Hz', '500 Hz', '1000 Hz', '2000 Hz', '4000 Hz']

for f in FREQS:
    df[f] = pd.to_numeric(df[f], errors="coerce")

# Bilinen/Bilinmeyen ayrımı
has_c80 = ~df[FREQS].isna().any(axis=1)
known_indices = np.where(has_c80)[0]
unknown_indices = np.where(~has_c80)[0]

print(f"   📍 Toplam koltuk: {len(df)}")
print(f"   ✅ Ölçülmüş koltuk: {len(known_indices)} ({100*len(known_indices)/len(df):.1f}%)")
print(f"   ❓ Tahmin edilecek: {len(unknown_indices)} ({100*len(unknown_indices)/len(df):.1f}%)")
print(f"   🎵 Frekans bantları: {len(FREQS)} adet")

In [ ]:
# =============================================================================
# HÜCRE 3: FEATURE MÜHENDİSLİĞİ
# =============================================================================

print("\n" + "=" * 70)
print("🔧 FEATURE MÜHENDİSLİĞİ")
print("=" * 70)

# Sahne koordinatı
stage = df.iloc[180][["X", "Y", "Z"]].values.astype(float)

# 1. Geometrik özellikler
df["dx"] = df["X"] - stage[0]
df["dy"] = df["Y"] - stage[1]
df["dz"] = df["Z"] - stage[2]
df["distance"] = np.sqrt(df["dx"]**2 + df["dy"]**2 + df["dz"]**2)
df["log_distance"] = np.log(df["distance"] + 1e-6)
df["horizontal_dist"] = np.sqrt(df["dx"]**2 + df["dy"]**2)
df["angle_to_stage"] = np.arctan2(df["dy"], df["dx"])
df["height_ratio"] = df["dz"] / (df["distance"] + 1e-6)
df["log_horizontal"] = np.log(df["horizontal_dist"] + 1e-6)

# 2. Kategorik özellikler
df["distance_category"] = pd.cut(df["distance"], bins=[0, 5, 8, 15], labels=[0, 1, 2]).cat.codes.fillna(-1).astype(int)
df["region_x"] = pd.cut(df["X"], bins=5, labels=False).fillna(-1).astype(int)
df["region_y"] = pd.cut(df["Y"], bins=5, labels=False).fillna(-1).astype(int)
df["region_code"] = df["region_x"] * 5 + df["region_y"]

# 3. Duvar özellikleri
bounds = {"x_min": df["X"].min(), "x_max": df["X"].max(), 
          "y_min": df["Y"].min(), "y_max": df["Y"].max()}
df["dist_left_wall"] = df["X"] - bounds["x_min"]
df["dist_right_wall"] = bounds["x_max"] - df["X"]
df["dist_front_wall"] = df["Y"] - bounds["y_min"]
df["dist_back_wall"] = bounds["y_max"] - df["Y"]
df["dist_to_wall"] = df[["dist_left_wall", "dist_right_wall", "dist_front_wall", "dist_back_wall"]].min(axis=1)
df["near_wall"] = (df["dist_to_wall"] < 2.0).astype(int)
df["vertical_angle"] = np.arctan2(df["dz"], df["horizontal_dist"])
df["wall_reflection"] = 1.0 / (df["dist_to_wall"] + 0.1)
df["corner_effect"] = ((df["dist_left_wall"] < 1.5).astype(int) + 
                        (df["dist_right_wall"] < 1.5).astype(int) +
                        (df["dist_front_wall"] < 1.5).astype(int) +
                        (df["dist_back_wall"] < 1.5).astype(int))
df["min_two_wall_dist"] = df[["dist_left_wall", "dist_right_wall", 
                               "dist_front_wall", "dist_back_wall"]].apply(lambda r: sum(sorted(r)[:2]), axis=1)

# 4. Graf özellikleri
coords = df[["X", "Y", "Z"]].values.astype(float)
dist_matrix = cdist(coords, coords, metric='euclidean')
NEIGHBOR_THRESHOLD = 2.5
adjacency = (dist_matrix < NEIGHBOR_THRESHOLD) & (dist_matrix > 0)

df["neighbor_count"] = adjacency.sum(axis=1)
df["known_neighbor_count"] = [sum(1 for n in np.where(adjacency[i])[0] if n in known_indices) for i in range(len(df))]
df["dist_to_center"] = np.sqrt((df["X"] - df["X"].mean())**2 + (df["Y"] - df["Y"].mean())**2)
df["x_normalized"] = (df["X"] - df["X"].min()) / (df["X"].max() - df["X"].min())
df["y_normalized"] = (df["Y"] - df["Y"].min()) / (df["Y"].max() - df["Y"].min())

# Feature listesi (21 adet)
FEATURE_COLS = [
    "log_distance", "dx", "dy", "dz", "horizontal_dist", "angle_to_stage", "height_ratio", "log_horizontal",
    "distance_category", "region_code", "dist_to_wall", "near_wall", "vertical_angle",
    "wall_reflection", "corner_effect", "min_two_wall_dist",
    "neighbor_count", "known_neighbor_count", "dist_to_center", "x_normalized", "y_normalized"
]

print(f"   ✅ Toplam {len(FEATURE_COLS)} özellik oluşturuldu")
print(f"   📐 Geometrik: 8, Kategorik: 2, Duvar: 6, Graf: 5")

In [ ]:
# =============================================================================
# HÜCRE 4: VERİ HAZIRLAMA VE TENSOR DÖNÜŞÜMÜ
# =============================================================================

print("\n" + "=" * 70)
print("📦 VERİ HAZIRLAMA")
print("=" * 70)

# Feature matrisi
all_features = df[FEATURE_COLS].values.astype(float)
all_features = np.nan_to_num(all_features, nan=0.0)

# Normalizasyon
feature_scaler = StandardScaler()
all_features_scaled = feature_scaler.fit_transform(all_features)

# C80 değerleri
all_c80 = df[FREQS].values.astype(float)
all_c80 = np.nan_to_num(all_c80, nan=0.0)

c80_scaler = StandardScaler()
c80_scaler.fit(all_c80[known_indices])
all_c80_scaled = c80_scaler.transform(all_c80)

# Tensor dönüşümü
features_t = torch.tensor(all_features_scaled, dtype=torch.float32, device=device)
c80_t = torch.tensor(all_c80_scaled, dtype=torch.float32, device=device)
dist_matrix_t = torch.tensor(dist_matrix, dtype=torch.float32, device=device)
known_mask = has_c80.values
known_mask_t = torch.tensor(known_mask, dtype=torch.bool, device=device)

print(f"   ✅ Feature boyutu: {features_t.shape}")
print(f"   ✅ C80 boyutu: {c80_t.shape}")
print(f"   ✅ Mesafe matrisi: {dist_matrix_t.shape}")

In [ ]:
# =============================================================================
# HÜCRE 5: BİDİRECTİONAL RNN MODELİ
# =============================================================================

class BidirectionalC80Model(nn.Module):
    """
    Bidirectional LSTM + MLP Hybrid Model
    - İlk koltuktan sona, sondan başa tahmin
    - Feature-based ve IDW tabanlı fusion
    """
    def __init__(self, feature_dim, hidden_dim=128, num_layers=2):
        super().__init__()
        
        # Feature encoder
        self.feature_encoder = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        # Bidirectional LSTM
        self.bilstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0.2
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 6)  # 6 frekans
        )
        
        # Fusion weight
        self.fusion_alpha = nn.Parameter(torch.tensor(0.5))
    
    def forward(self, features, dist_matrix, known_mask, known_c80):
        batch_size = features.shape[0]
        
        # Feature encoding
        encoded = self.feature_encoder(features)  # (N, hidden)
        
        # Reshape for LSTM: (1, N, hidden) - tüm koltuklar bir sequence
        lstm_input = encoded.unsqueeze(0)
        
        # Bidirectional LSTM
        lstm_out, _ = self.bilstm(lstm_input)  # (1, N, hidden*2)
        lstm_out = lstm_out.squeeze(0)  # (N, hidden*2)
        
        # Decode to predictions
        feat_pred = self.decoder(lstm_out)  # (N, 6)
        
        # IDW interpolation
        eps = 0.3
        weights = 1.0 / (dist_matrix ** 1.5 + eps)
        weights = weights * known_mask.float().unsqueeze(0)
        weights_norm = weights / weights.sum(dim=1, keepdim=True).clamp(min=1e-6)
        idw_pred = torch.mm(weights_norm, known_c80)
        
        # Adaptive fusion
        alpha = torch.sigmoid(self.fusion_alpha)
        final_pred = alpha * feat_pred + (1 - alpha) * idw_pred
        
        # Bilinen değerleri koru
        output = torch.where(known_mask.unsqueeze(-1), known_c80, final_pred)
        
        return output, feat_pred, idw_pred, alpha

print("✅ BidirectionalC80Model tanımlandı")

In [ ]:
# =============================================================================
# HÜCRE 6: TRAIN/TEST DEĞERLENDİRME
# =============================================================================

print("\n" + "=" * 70)
print("🧪 TRAIN/TEST DEĞERLENDİRME (63 koltuk → %80/%20)")
print("=" * 70)

# Train/Test split
np.random.seed(42)
train_idx, test_idx = train_test_split(known_indices, test_size=0.2, random_state=42)
print(f"   📚 Eğitim: {len(train_idx)} koltuk")
print(f"   🧪 Test: {len(test_idx)} koltuk")

# Train mask
train_mask = np.zeros(len(df), dtype=bool)
train_mask[train_idx] = True
train_mask_t = torch.tensor(train_mask, dtype=torch.bool, device=device)

# Train C80 (test değerleri gizle)
train_c80_t = c80_t.clone()
train_c80_t[test_idx] = 0.0

# Ensemble eğitimi
N_MODELS = 5
N_EPOCHS = 300
ensemble_models = []

print(f"\n🔄 Ensemble eğitimi ({N_MODELS} model)...")

for m in range(N_MODELS):
    torch.manual_seed(42 + m * 10)
    np.random.seed(42 + m * 10)
    
    model = BidirectionalC80Model(feature_dim=21, hidden_dim=128, num_layers=2).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.003, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=N_EPOCHS, eta_min=1e-5)
    criterion = nn.HuberLoss(delta=1.0)
    
    best_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(N_EPOCHS):
        model.train()
        optimizer.zero_grad()
        
        output, feat_pred, idw_pred, alpha = model(features_t, dist_matrix_t, train_mask_t, train_c80_t)
        loss = criterion(feat_pred[train_mask_t], c80_t[train_mask_t])
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        
        if loss.item() < best_loss:
            best_loss = loss.item()
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= 40 and epoch > 100:
            break
    
    ensemble_models.append(model)
    print(f"   ✅ Model {m+1}/{N_MODELS}: Loss = {best_loss:.4f}, Epoch = {epoch+1}")

# Test tahminleri
print("\n📊 Test sonuçları hesaplanıyor...")
ensemble_preds = []
for model in ensemble_models:
    model.eval()
    with torch.no_grad():
        out, _, _, _ = model(features_t, dist_matrix_t, train_mask_t, train_c80_t)
        ensemble_preds.append(out.cpu().numpy())

ensemble_mean = np.mean(ensemble_preds, axis=0)

# Test metrikleri
test_pred = c80_scaler.inverse_transform(ensemble_mean[test_idx])
test_true = c80_scaler.inverse_transform(c80_t[test_idx].cpu().numpy())

test_r2 = r2_score(test_true.flatten(), test_pred.flatten())
test_mae = mean_absolute_error(test_true.flatten(), test_pred.flatten())
test_rmse = np.sqrt(mean_squared_error(test_true.flatten(), test_pred.flatten()))

print("\n" + "=" * 50)
print("📈 TEST SONUÇLARI")
print("=" * 50)
print(f"   R² Score:  {test_r2:.4f}")
print(f"   MAE:       {test_mae:.3f} dB")
print(f"   RMSE:      {test_rmse:.3f} dB")

# Frekans bazlı
print("\n📊 Frekans Bazlı Performans:")
print("-" * 50)
freq_metrics = {}
for i, freq in enumerate(FREQS):
    r2 = r2_score(test_true[:, i], test_pred[:, i])
    mae = mean_absolute_error(test_true[:, i], test_pred[:, i])
    freq_metrics[freq] = {'r2': r2, 'mae': mae}
    print(f"   {freq:5d} Hz: R² = {r2:.3f}, MAE = {mae:.2f} dB")

# Başarı kontrolü
if test_r2 >= 0.90:
    print("\n✅ Model başarılı! (R² >= 0.90) Tüm salon tahminine geçiliyor...")
    MODEL_SUCCESS = True
else:
    print("\n⚠️ Model R² < 0.90, iyileştirme gerekebilir.")
    MODEL_SUCCESS = False

In [ ]:
# =============================================================================
# HÜCRE 7: TÜM SALON TAHMİNİ
# =============================================================================

print("\n" + "=" * 70)
print("🎯 TÜM SALON TAHMİNİ (Bidirectional RNN)")
print("=" * 70)

# Tüm verilerle tahmin
full_ensemble_preds = []
for model in ensemble_models:
    model.eval()
    with torch.no_grad():
        out, _, _, alpha = model(features_t, dist_matrix_t, known_mask_t, c80_t)
        full_ensemble_preds.append(out.cpu().numpy())

full_ensemble_mean = np.mean(full_ensemble_preds, axis=0)
final_predictions = c80_scaler.inverse_transform(full_ensemble_mean)

print(f"   📍 Fusion Alpha (öğrenilen): {alpha.item():.3f}")
print(f"   📊 Feature ağırlığı: {alpha.item()*100:.1f}%")
print(f"   📊 IDW ağırlığı: {(1-alpha.item())*100:.1f}%")

# DataFrame'e ekle
df_results = df.copy()
for i, freq in enumerate(FREQS):
    df_results[f"{freq}_pred"] = final_predictions[:, i]
    df_results[f"{freq}_error"] = np.nan
    df_results.loc[has_c80, f"{freq}_error"] = final_predictions[has_c80, i] - df_results.loc[has_c80, freq]

# Tahmin istatistikleri
print("\n📊 Bilinmeyen Koltuklar Tahmin Aralıkları:")
print("-" * 60)
for freq in FREQS:
    vals = df_results.loc[~has_c80, f"{freq}_pred"]
    print(f"   {freq:5d} Hz: min={vals.min():.1f}, max={vals.max():.1f}, mean={vals.mean():.1f} dB")

# Doğrulama
print("\n📊 Bilinen Koltuklar Doğrulama:")
print("-" * 60)
for freq in FREQS:
    true_vals = df_results.loc[has_c80, freq].values
    pred_vals = df_results.loc[has_c80, f"{freq}_pred"].values
    r2 = r2_score(true_vals, pred_vals)
    mae = np.abs(true_vals - pred_vals).mean()
    print(f"   {freq:5d} Hz: R²={r2:.3f}, MAE={mae:.2f} dB")

In [ ]:
# =============================================================================
# HÜCRE 8: PROFESYONELGRAFİKLER - PERFORMANS ANALİZİ
# =============================================================================

print("\n" + "=" * 70)
print("📊 PERFORMANS GRAFİKLERİ")
print("=" * 70)

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('TÜBİTAK 1001 - C80 Tahmin Modeli Performans Analizi', fontsize=16, fontweight='bold')

# 1. Frekans Bazlı R² Skoru
ax1 = axes[0, 0]
r2_values = [freq_metrics[f]['r2'] for f in FREQS]
colors = ['#2ecc71' if r > 0.95 else '#f39c12' if r > 0.90 else '#e74c3c' for r in r2_values]
bars = ax1.bar(FREQ_LABELS, r2_values, color=colors, edgecolor='black', linewidth=1.2)
ax1.axhline(y=0.95, color='green', linestyle='--', linewidth=2, label='Mükemmel (0.95)')
ax1.axhline(y=0.90, color='orange', linestyle='--', linewidth=2, label='İyi (0.90)')
ax1.set_ylabel('R² Skoru', fontweight='bold')
ax1.set_title('Frekans Bazlı R² Performansı', fontweight='bold')
ax1.set_ylim(0.8, 1.0)
ax1.legend(loc='lower right')
for bar, val in zip(bars, r2_values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005, f'{val:.3f}', 
             ha='center', va='bottom', fontweight='bold', fontsize=10)

# 2. Frekans Bazlı MAE
ax2 = axes[0, 1]
mae_values = [freq_metrics[f]['mae'] for f in FREQS]
colors2 = ['#2ecc71' if m < 0.3 else '#f39c12' if m < 0.5 else '#e74c3c' for m in mae_values]
bars2 = ax2.bar(FREQ_LABELS, mae_values, color=colors2, edgecolor='black', linewidth=1.2)
ax2.axhline(y=0.3, color='green', linestyle='--', linewidth=2, label='Mükemmel (<0.3 dB)')
ax2.axhline(y=0.5, color='orange', linestyle='--', linewidth=2, label='İyi (<0.5 dB)')
ax2.set_ylabel('MAE (dB)', fontweight='bold')
ax2.set_title('Frekans Bazlı Ortalama Mutlak Hata', fontweight='bold')
ax2.legend(loc='upper right')
for bar, val in zip(bars2, mae_values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, f'{val:.2f}', 
             ha='center', va='bottom', fontweight='bold', fontsize=10)

# 3. Gerçek vs Tahmin Scatter Plot
ax3 = axes[0, 2]
ax3.scatter(test_true.flatten(), test_pred.flatten(), alpha=0.6, c='#3498db', edgecolors='black', s=50)
min_val = min(test_true.min(), test_pred.min())
max_val = max(test_true.max(), test_pred.max())
ax3.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='İdeal')
ax3.set_xlabel('Gerçek C80 (dB)', fontweight='bold')
ax3.set_ylabel('Tahmin C80 (dB)', fontweight='bold')
ax3.set_title(f'Gerçek vs Tahmin (R² = {test_r2:.3f})', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Hata Dağılımı Histogram
ax4 = axes[1, 0]
errors = (test_pred - test_true).flatten()
ax4.hist(errors, bins=30, color='#9b59b6', edgecolor='black', alpha=0.7)
ax4.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax4.axvline(x=errors.mean(), color='green', linestyle='-', linewidth=2, label=f'Ortalama: {errors.mean():.2f} dB')
ax4.set_xlabel('Tahmin Hatası (dB)', fontweight='bold')
ax4.set_ylabel('Frekans', fontweight='bold')
ax4.set_title('Hata Dağılımı', fontweight='bold')
ax4.legend()

# 5. Test Koltukları Hata Analizi
ax5 = axes[1, 1]
seat_errors = np.abs(test_pred - test_true).sum(axis=1)
colors5 = ['#2ecc71' if e < 2 else '#f39c12' if e < 4 else '#e74c3c' for e in seat_errors]
ax5.barh(range(len(test_idx)), seat_errors, color=colors5, edgecolor='black')
ax5.set_yticks(range(len(test_idx)))
ax5.set_yticklabels([f'Koltuk {i}' for i in test_idx])
ax5.set_xlabel('Toplam Hata (dB) - 6 Frekans', fontweight='bold')
ax5.set_title('Test Koltukları Hata Analizi', fontweight='bold')
ax5.axvline(x=3, color='orange', linestyle='--', linewidth=2)

# 6. Model Özet Metrikleri
ax6 = axes[1, 2]
ax6.axis('off')
summary_text = f"""
╔══════════════════════════════════════════╗
║     MODEL PERFORMANS ÖZETİ               ║
╠══════════════════════════════════════════╣
║  Test R² Skoru:     {test_r2:.4f}              ║
║  Test MAE:          {test_mae:.3f} dB              ║
║  Test RMSE:         {test_rmse:.3f} dB              ║
╠══════════════════════════════════════════╣
║  Eğitim Koltuk:     {len(train_idx)}                   ║
║  Test Koltuk:       {len(test_idx)}                    ║
║  Tahmin Edilecek:   {len(unknown_indices)}                  ║
╠══════════════════════════════════════════╣
║  Model:      Bidirectional LSTM          ║
║  Ensemble:   {N_MODELS} model                       ║
║  Frekans:    6 oktav band                ║
╚══════════════════════════════════════════╝
"""
ax6.text(0.1, 0.5, summary_text, fontsize=12, fontfamily='monospace', 
         verticalalignment='center', bbox=dict(boxstyle='round', facecolor='#ecf0f1', alpha=0.8))

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('C80_Performans_Analizi.png', dpi=200, bbox_inches='tight', facecolor='white')
plt.show()
print("💾 Grafik kaydedildi: C80_Performans_Analizi.png")

In [ ]:
# =============================================================================
# HÜCRE 9: SALON HARİTALARI
# =============================================================================

print("\n" + "=" * 70)
print("🗺️ SALON C80 HARİTALARI")
print("=" * 70)

def plot_salon_map(df_pred, freq, save=True):
    fig, axes = plt.subplots(1, 3, figsize=(20, 7))
    fig.suptitle(f'{freq} Hz - C80 Salon Haritası', fontsize=16, fontweight='bold')
    
    known = df_pred[df_pred[freq].notna()]
    unknown = df_pred[df_pred[freq].isna()]
    
    vmin = df_pred[f"{freq}_pred"].min() - 0.5
    vmax = df_pred[f"{freq}_pred"].max() + 0.5
    
    # 1. Ölçülen
    sc1 = axes[0].scatter(known["X"], known["Y"], c=known[freq], 
                          cmap='RdYlGn', s=100, edgecolors='black', linewidth=0.8,
                          vmin=vmin, vmax=vmax)
    axes[0].scatter(unknown["X"], unknown["Y"], c='lightgray', s=40, alpha=0.5, marker='s')
    axes[0].scatter([0], [0], c='red', s=200, marker='*', label='Sahne', zorder=5)
    plt.colorbar(sc1, ax=axes[0], label='C80 (dB)', shrink=0.8)
    axes[0].set_title(f'Ölçülen C80\n({len(known)} koltuk)', fontweight='bold')
    axes[0].set_xlabel('X (m)'); axes[0].set_ylabel('Y (m)')
    axes[0].legend(loc='upper right')
    
    # 2. Tahmin edilen (tüm salon)
    sc2 = axes[1].scatter(df_pred["X"], df_pred["Y"], c=df_pred[f"{freq}_pred"],
                          cmap='RdYlGn', s=60, edgecolors='black', linewidth=0.5,
                          vmin=vmin, vmax=vmax)
    axes[1].scatter([0], [0], c='red', s=200, marker='*', zorder=5)
    plt.colorbar(sc2, ax=axes[1], label='C80 Tahmin (dB)', shrink=0.8)
    axes[1].set_title(f'Tahmin C80\n({len(df_pred)} koltuk)', fontweight='bold')
    axes[1].set_xlabel('X (m)'); axes[1].set_ylabel('Y (m)')
    
    # 3. Hata haritası
    error_data = df_pred[df_pred[f"{freq}_error"].notna()]
    sc3 = axes[2].scatter(error_data["X"], error_data["Y"], 
                          c=error_data[f"{freq}_error"],
                          cmap='coolwarm', s=100, edgecolors='black', linewidth=0.8,
                          vmin=-1.5, vmax=1.5)
    axes[2].scatter(unknown["X"], unknown["Y"], c='lightgray', s=40, alpha=0.3, marker='s')
    plt.colorbar(sc3, ax=axes[2], label='Hata (dB)', shrink=0.8)
    axes[2].set_title(f'Tahmin Hatası\n(Mavi=düşük, Kırmızı=yüksek)', fontweight='bold')
    axes[2].set_xlabel('X (m)'); axes[2].set_ylabel('Y (m)')
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    if save:
        plt.savefig(f'C80_Salon_{freq}Hz.png', dpi=200, bbox_inches='tight', facecolor='white')
    plt.show()

# Haritaları oluştur
for freq in [500, 1000, 2000]:
    plot_salon_map(df_results, freq)
    print(f"💾 Kaydedildi: C80_Salon_{freq}Hz.png")

In [ ]:
# =============================================================================
# HÜCRE 10: SONUÇ TABLOSU VE EXCEL ÇIKTISI
# =============================================================================

print("\n" + "=" * 70)
print("📋 SONUÇ TABLOSU VE ÇIKTILAR")
print("=" * 70)

# Sonuç tablosu
result_summary = pd.DataFrame({
    'Frekans (Hz)': FREQS,
    'Test R²': [freq_metrics[f]['r2'] for f in FREQS],
    'Test MAE (dB)': [freq_metrics[f]['mae'] for f in FREQS],
    'Tahmin Min (dB)': [df_results.loc[~has_c80, f"{f}_pred"].min() for f in FREQS],
    'Tahmin Max (dB)': [df_results.loc[~has_c80, f"{f}_pred"].max() for f in FREQS],
    'Tahmin Ort (dB)': [df_results.loc[~has_c80, f"{f}_pred"].mean() for f in FREQS]
})

print("\n📊 Frekans Bazlı Özet Tablo:")
print(result_summary.to_string(index=False))

# Excel kaydet
output_cols = ['seat_id', 'X', 'Y', 'Z'] + FREQS + [f"{f}_pred" for f in FREQS] + [f"{f}_error" for f in FREQS]
df_output = df_results[output_cols].copy()
df_output.to_excel('C80_Final_Tahminler.xlsx', index=False)
print(f"\n💾 Tahminler kaydedildi: C80_Final_Tahminler.xlsx")

# Model kaydet
torch.save({
    'ensemble_models': [m.state_dict() for m in ensemble_models],
    'feature_scaler': feature_scaler,
    'c80_scaler': c80_scaler,
    'test_r2': test_r2,
    'test_mae': test_mae,
    'feature_cols': FEATURE_COLS,
    'freqs': FREQS
}, 'C80_Final_Model.pth')
print(f"💾 Model kaydedildi: C80_Final_Model.pth")

In [ ]:
# =============================================================================
# HÜCRE 11: FİNAL RAPOR
# =============================================================================

print("\n")
print("╔" + "═" * 68 + "╗")
print("║" + " " * 20 + "TÜBİTAK 1001 PROJESİ" + " " * 28 + "║")
print("║" + " " * 10 + "Konser Salonu C80 Akustik Tahmin Modeli" + " " * 18 + "║")
print("║" + " " * 25 + "FİNAL RAPORU" + " " * 31 + "║")
print("╠" + "═" * 68 + "╣")
print("║" + " " * 68 + "║")
print(f"║  📊 VERİ SETİ                                                      ║")
print(f"║     • Toplam koltuk:         {len(df):>5}                                 ║")
print(f"║     • Ölçülmüş koltuk:       {len(known_indices):>5} ({100*len(known_indices)/len(df):.1f}%)                         ║")
print(f"║     • Tahmin edilen:         {len(unknown_indices):>5} ({100*len(unknown_indices)/len(df):.1f}%)                         ║")
print(f"║     • Frekans bandı:         {len(FREQS):>5} (125-4000 Hz)                    ║")
print("║" + " " * 68 + "║")
print(f"║  🧠 MODEL MİMARİSİ                                                 ║")
print(f"║     • Tip:                   Bidirectional LSTM + MLP              ║")
print(f"║     • Ensemble:              {N_MODELS} model                                 ║")
print(f"║     • Feature sayısı:        {len(FEATURE_COLS)}                                    ║")
print(f"║     • Hidden dim:            128                                   ║")
print("║" + " " * 68 + "║")
print(f"║  📈 PERFORMANS (Test Seti: {len(test_idx)} koltuk)                              ║")
print(f"║     • R² Skoru:              {test_r2:.4f}                                 ║")
print(f"║     • MAE:                   {test_mae:.3f} dB                                ║")
print(f"║     • RMSE:                  {test_rmse:.3f} dB                                ║")
print("║" + " " * 68 + "║")
print(f"║  📁 ÇIKTI DOSYALARI                                                ║")
print(f"║     • C80_Final_Tahminler.xlsx (Tüm tahminler)                     ║")
print(f"║     • C80_Final_Model.pth (Eğitilmiş model)                        ║")
print(f"║     • C80_Performans_Analizi.png (Grafikler)                       ║")
print(f"║     • C80_Salon_500Hz.png, 1000Hz.png, 2000Hz.png (Haritalar)      ║")
print("║" + " " * 68 + "║")
print("╚" + "═" * 68 + "╝")
print("\n✅ Proje başarıyla tamamlandı!")